In [ ]:
!pip install numpy
!pip install pandas

  Using cached BeautifulSoup-3.2.2.tar.gz (32 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'error'


  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [22 lines of output]
      Traceback (most recent call last):
        File "d:\Р’РЈР—\РёРЅР¶РµРЅРёСЂРµРЅРі РґР°РЅРЅС‹С…\github\DataEngineeringHomework\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 353, in <module>
          main()
        File "d:\Р’РЈР—\РёРЅР¶РµРЅРёСЂРµРЅРі РґР°РЅРЅС‹С…\github\DataEngineeringHomework\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "d:\Р’РЈР—\РёРЅР¶РµРЅРёСЂРµРЅРі РґР°РЅРЅС‹С…\github\DataEngineeringHomework\.venv\Lib\site-packages\pip\_vendor\pyproject_hooks\_in_process\_in_process.py", line 118, in get_requires_for_build_wheel
          return hook(config_settings)
                 ^^^^^^^^^^^^^^^

***Вариант №36***

Задание №1

In [67]:
import pandas as pd
import sqlite3

f = './data/1-2/item.msgpack'

df = pd.read_json(f)


# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect('data.db')
cursor = conn.cursor()

# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS data (
	id INTEGER,
	name TEXT,
	street TEXT,
	city TEXT,
	zipcode INTEGER,
	floors INTEGER,
	year INTEGER,
	parking BOOLEAN,
	prob_price INTEGER,
	views INTEGER
)
''')

df.to_sql('data', conn, if_exists='replace', index=False)

conn.commit()

VAR = 5

query1 = f'SELECT * FROM data ORDER BY prob_price LIMIT {VAR + 10}'
result1 = pd.read_sql_query(query1, conn)
result1.to_json('1_output_sorted.json', orient='records', force_ascii=False)
query2 = 'SELECT SUM(prob_price), MIN(prob_price), MAX(prob_price), AVG(prob_price) FROM data'
result2 = cursor.execute(query2).fetchall()
print("Sum, Min, Max, Average of prob_price:", result2)
query3 = 'SELECT city, COUNT(*) as frequency FROM data GROUP BY city'
result3 = pd.read_sql_query(query3, conn)
print("Frequency of cities:\n", result3)
query4 = f'SELECT * FROM data WHERE floors > 5 ORDER BY prob_price LIMIT {VAR + 10}'
result4 = pd.read_sql_query(query4, conn)
result4.to_json('1_output_filtered_sorted.json', orient='records', force_ascii=False)

conn.close()


Sum, Min, Max, Average of prob_price: [(42363043461, 23927354, 994057682, 516622481.23170733)]
Frequency of cities:
                          city  frequency
0                    Аликанте          2
1                    Алма-Ата          1
2           Алькала-де-Энарес          2
3                     Артейхо          1
4                        Баку          2
5                      Бишкек          1
6                    Будапешт          1
7                  Вальядолид          2
8                     Варшава          1
9                        Виго          2
10                    Вильнюс          1
11                    Гранада          1
12                     Ереван          2
13                     Загреб          1
14                   Камбадос          2
15                  Картахена          1
16                    Касерес          2
17                    Кишинев          2
18                     Краков          1
19                     Куэнка          1
20                  Ла

Задание №2

In [31]:
import pandas as pd
import sqlite3

subitem_df = pd.read_pickle("./data/1-2/subitem.pkl")

if isinstance(subitem_df, list):
    subitem_df = pd.DataFrame(subitem_df)

print("Columns in subitem_df:", subitem_df.columns)

conn = sqlite3.connect("data.db")
cursor = conn.cursor()

cursor.execute(
    """
CREATE TABLE IF NOT EXISTS subitem (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT,
    rating INTEGER,
    value INTEGER,
    security INTEGER,
    functionality INTEGER,
    comment TEXT,
    dataId INTEGER
)
"""
)

data_df = pd.read_sql_query('SELECT id, name FROM data', conn)
name_to_id = dict(zip(data_df['name'], data_df['id']))
subitem_df['dataId'] = subitem_df['name'].map(name_to_id)
subitem_df.to_sql('subitem', conn, if_exists='replace', index=False)


query1 = '''
SELECT d.name, s.comment
FROM data d
JOIN subitem s ON d.id = s.dataId
'''
result1 = pd.read_sql_query(query1, conn)
print("Query 1 Result:\n", result1)

query2 = '''
SELECT d.city, AVG(s.rating) as avg_rating
FROM data d
JOIN subitem s ON d.id = s.dataId
GROUP BY d.city
'''
result2 = pd.read_sql_query(query2, conn)
print("Query 2 Result:\n", result2)

conn.close()

Columns in subitem_df: Index(['name', 'rating', 'convenience', 'security', 'functionality',
       'comment'],
      dtype='object')
Query 1 Result:
            name                                            comment
0       Кафе 61                  Мне нравится дизайн этого здания.
1       Кафе 61                  Мне нравится дизайн этого здания.
2       Кафе 61                  Мне нравится дизайн этого здания.
3       Кафе 61                  Мне нравится дизайн этого здания.
4       Кафе 61               У этого здания отличная архитектура.
...         ...                                                ...
6162  Гнездо 81  Это здание является примером функционального д...
6163  Гнездо 81  Это здание является примером функционального д...
6164  Гнездо 81  Это здание является примером функционального д...
6165  Гнездо 81  Это здание является примером функционального д...
6166  Гнездо 81  Это здание является примером функционального д...

[6167 rows x 2 columns]
Query 2 Result:
     

Задание №3

In [55]:
import pandas as pd
import json

# Открытие файла
file_path_text = "./data/3/_part_1.text"
with open(file_path_text, "r", encoding="utf-8") as f:
    data = f.read()

# Разделение на записи
records = data.split("=====")

# Парсинг записей
parsed_data = []
for record in records:
    if record.strip():  # Игнорируем пустые строки
        record_dict = {}
        for line in record.strip().split("\n"):
            if "::" in line:
                key, value = line.split("::", 1)
                record_dict[key.strip()] = value.strip()
        parsed_data.append(record_dict)

# Преобразование в DataFrame
df_text = pd.DataFrame(parsed_data)

file_path_pkl = "./data/3/_part_2.pkl"
df_pkl = pd.DataFrame(pd.read_pickle(file_path_pkl))

conn = sqlite3.connect("music.db")
cursor = conn.cursor()

df_text.to_sql("music", conn, if_exists="replace", index=False)
df_pkl.to_sql("music", conn, if_exists="replace", index=False)

conn.commit()

VAR = 5
cursor.execute("SELECT * FROM music ORDER BY tempo LIMIT ?", (VAR + 10,))
rows = cursor.fetchall()
with open("3_output.json", "w") as f:
    json.dump(rows, f)

cursor.execute("SELECT SUM(tempo), MIN(tempo), MAX(tempo), AVG(tempo) FROM music")
print(cursor.fetchall())

cursor.execute("SELECT genre, COUNT(*) FROM music GROUP BY genre")
print(cursor.fetchall())

conn.close()

[(96785.764, '100.0', '99.993', 119.4885975308642)]
[('Dance/Electronic', 17), ('Folk/Acoustic, rock', 1), ('R&B', 2), ('World/Traditional, hip hop', 1), ('World/Traditional, pop, Folk/Acoustic', 1), ('World/Traditional, rock, pop', 1), ('country', 4), ('country, latin', 1), ('hip hop', 53), ('hip hop, Dance/Electronic', 5), ('hip hop, R&B', 1), ('hip hop, latin, Dance/Electronic', 1), ('hip hop, pop', 119), ('hip hop, pop, Dance/Electronic', 31), ('hip hop, pop, R&B', 98), ('hip hop, pop, R&B, Dance/Electronic', 2), ('hip hop, pop, R&B, latin', 2), ('hip hop, pop, country', 1), ('hip hop, pop, latin', 6), ('hip hop, pop, rock', 4), ('hip hop, rock, pop', 1), ('latin', 5), ('metal', 4), ('pop', 164), ('pop, Dance/Electronic', 90), ('pop, Folk/Acoustic', 5), ('pop, R&B', 78), ('pop, R&B, Dance/Electronic', 1), ('pop, R&B, easy listening', 1), ('pop, country', 1), ('pop, easy listening, jazz', 1), ('pop, latin', 14), ('pop, rock', 7), ('pop, rock, Dance/Electronic', 1), ('pop, rock, meta

Задание №4

In [5]:
import sqlite3
import msgpack
import json
import os

# Пути к файлам
product_data_path = "./data/4/_product_data.msgpack"
update_data_path = "./data/4/_update_data.json"
db_path = "products.db"


# 1. Создание базы данных и таблиц
def create_database():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    cursor.execute("DROP TABLE IF EXISTS products;")
    cursor.execute("DROP TABLE IF EXISTS product_updates;")

    cursor.execute(
        """
    CREATE TABLE IF NOT EXISTS products (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        price REAL NOT NULL CHECK (price >= 0),
        quantity INTEGER NOT NULL CHECK (quantity >= 0),
        category TEXT,
        from_city TEXT,
        is_available BOOLEAN NOT NULL,
        views INTEGER NOT NULL DEFAULT 0,
        update_counter INTEGER NOT NULL DEFAULT 0
    );
    """
    )

    conn.commit()
    conn.close()


# 2. Импорт данных о товарах
def import_products():
    with open(product_data_path, "rb") as f:
        product_data = msgpack.unpack(f, raw=False)

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    insert_query = """
    INSERT INTO products (name, price, quantity, category, from_city, is_available, views)
    VALUES (?, ?, ?, ?, ?, ?, ?);
    """
    for product in product_data:
        cursor.execute(
            insert_query,
            (
                product.get("name", "Unknown"),
                product.get("price", 0.0),
                product.get("quantity", 0),
                product.get("category", "Uncategorized"),
                product.get("fromCity", "Unknown"),
                product.get("isAvailable", False),
                product.get("views", 0),
            ),
        )

    conn.commit()
    conn.close()


# 3. Применение обновлений
def apply_updates():
    with open(update_data_path, "r") as f:
        update_data = json.load(f)

    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    for update in update_data:
        name = update["name"]
        method = update["method"]
        param = update["param"]
        try:
            cursor.execute("BEGIN")

            cursor.execute("SELECT id FROM products WHERE name = ?", (name,))
            result = cursor.fetchone()
            if not result:
                # print(
                #     f"Товар '{name}' отсутствует в таблице products. Обновление пропущено."
                # )
                cursor.execute("ROLLBACK")
                continue
            product_id = result[0]


            if method == "price_abs":
                cursor.execute(
                    """
                UPDATE products
                SET price = price + ?, update_counter = update_counter + 1
                WHERE id = ? AND price + ? >= 0;
                """,
                    (param, product_id, param),
                )
            elif method == "price_percent":
                cursor.execute(
                    """
                UPDATE products
                SET price = price * (1 + ?), update_counter = update_counter + 1
                WHERE id = ? AND price * (1 + ?) >= 0;
                """,
                    (param, product_id, param),
                )
            elif method == "quantity_add":
                cursor.execute(
                    """
                UPDATE products
                SET quantity = quantity + ?, update_counter = update_counter + 1
                WHERE id = ? AND quantity + ? >= 0;
                """,
                    (param, product_id, param),
                )
            elif method == "quantity_sub":
                cursor.execute(
                    """
                UPDATE products
                SET quantity = quantity + ?, update_counter = update_counter + 1
                WHERE id = ? AND quantity + ? >= 0;
                """,
                    (param, product_id, param),
                )
            elif method == "remove":
                cursor.execute("DELETE FROM products WHERE id = ?", (product_id,))
            elif method == "available":
                cursor.execute(
                    """
                UPDATE products
                SET is_available = ?, update_counter = update_counter + 1
                WHERE id = ?;
                """,
                    (param, product_id),
                )

            cursor.execute("COMMIT")
        except Exception as e:
            print(f"Ошибка при обновлении {name}: {e}")
            cursor.execute("ROLLBACK")

    conn.commit()
    conn.close()


# 4. Аналитические запросы
def perform_analysis():
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    print("Топ-10 самых обновляемых товаров:")
    cursor.execute(
        """
    SELECT name, update_counter
    FROM products
    ORDER BY update_counter DESC
    LIMIT 10;
    """
    )
    for row in cursor.fetchall():
        print(row)

    print("\nАнализ цен:")
    cursor.execute(
        """
    SELECT COUNT(*) AS product_count,
           SUM(price) AS total_price,
           MIN(price) AS min_price,
           MAX(price) AS max_price,
           AVG(price) AS avg_price
    FROM products;
    """
    )
    print(cursor.fetchone())

    print("\nАнализ остатков:")
    cursor.execute(
        """
    SELECT COUNT(*) AS product_count,
           SUM(quantity) AS total_quantity,
           MIN(quantity) AS min_quantity,
           MAX(quantity) AS max_quantity,
           AVG(quantity) AS avg_quantity
    FROM products;
    """
    )
    print(cursor.fetchone())

    conn.close()


create_database()
import_products()
apply_updates()
perform_analysis()

Топ-10 самых обновляемых товаров:
('magnificent grapes', 14)
('Logitech Wireless Keyboard', 14)
('luxurious hair mask', 13)
('magnificent manicure set', 12)
('elegant hair mask', 12)
('beautiful papaya', 11)
('pleasant saw', 10)
('pretty ladder', 9)
('luxurious plumbing', 8)
('delightful wallpaper', 8)

Анализ цен:
(49, 900755.7218743176, 9.17, 141384.74000000002, 18382.769834169747)

Анализ остатков:
(49, 27101, 13, 1117, 553.0816326530612)


Задание №5

In [8]:
import sqlite3
import csv
import json

# Соединение с базой данных (создаст новый файл базы данных)
conn = sqlite3.connect("library.db")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS authors;")
cursor.execute("DROP TABLE IF EXISTS genres;")
cursor.execute("DROP TABLE IF EXISTS books;")

# Создание таблиц
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS authors (
    author_id INT PRIMARY KEY,
    author_name VARCHAR(100) NOT NULL
);
"""
)
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS genres (
    genre_id INT PRIMARY KEY,
    genre_name VARCHAR(50) NOT NULL
);
"""
)
cursor.execute(
    """
CREATE TABLE IF NOT EXISTS books (
    book_id INT PRIMARY KEY,
    book_title VARCHAR(200) NOT NULL,
    author_id INT,
    genre_id INT,
    year_published INT,
    FOREIGN KEY (author_id) REFERENCES authors(author_id),
    FOREIGN KEY (genre_id) REFERENCES genres(genre_id)
);
"""
)

# Сохранение изменений и закрытие соединения
conn.commit()
conn.close()


# Функция для загрузки данных из CSV файла в таблицу
def load_csv_to_db(file_name, table_name):
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()

    with open(file_name, mode="r", encoding="utf-8") as file:
        reader = csv.reader(file)
        next(reader)  # Пропустить заголовок
        for row in reader:
            cursor.execute(
                f"INSERT INTO {table_name} VALUES ({','.join(['?']*len(row))})", row
            )

    conn.commit()
    conn.close()


# Загрузка данных
load_csv_to_db("./data/5/authors.csv", "authors")
load_csv_to_db("./data/5/genres.csv", "genres")
load_csv_to_db("./data/5/books.csv", "books")


def query_to_json(query):
    conn = sqlite3.connect("library.db")
    cursor = conn.cursor()
    cursor.execute(query)
    rows = cursor.fetchall()

    # Получаем имена столбцов
    columns = [description[0] for description in cursor.description]

    # Преобразуем результат в список словарей
    result = [dict(zip(columns, row)) for row in rows]
    conn.close()

    return json.dumps(result, indent=4)
    # Запись результата в JSON файл


# 1. Выборка с простым условием + сортировка + ограничение количество
query_1 = """
SELECT book_title, year_published FROM books
WHERE year_published > 1850
ORDER BY year_published DESC
LIMIT 3;
"""
print(query_to_json(query_1))

# 2. Подсчет количества книг по жанрам
query_2 = """
SELECT g.genre_name, COUNT(b.book_id) as book_count
FROM genres g
LEFT JOIN books b ON g.genre_id = b.genre_id
GROUP BY g.genre_name;
"""
print(query_to_json(query_2))

# 3. Группировка и агрегирование: Средний год публикации книг по авторам
query_3 = """
SELECT a.author_name, AVG(b.year_published) as avg_year
FROM authors a
JOIN books b ON a.author_id = b.author_id
GROUP BY a.author_name;
"""
print(query_to_json(query_3))

# 4. Выборка книг определенного автора
query_4 = """
SELECT book_title, year_published
FROM books
WHERE author_id = (SELECT author_id FROM authors WHERE author_name = 'Leo Tolstoy');
"""
print(query_to_json(query_4))

# 5. Подсчет количества книг по авторам
query_5 = """
SELECT a.author_name, COUNT(b.book_id) as book_count
FROM authors a
LEFT JOIN books b ON a.author_id = b.author_id
GROUP BY a.author_name;
"""
print(query_to_json(query_5))

# 6. Выборка книг по жанру
query_6 = """
SELECT book_title, year_published
FROM books
WHERE genre_id = (SELECT genre_id FROM genres WHERE genre_name = 'Science Fiction');
"""
print(query_to_json(query_6))

# 7. Средний год публикации книг по жанрам
query_7 = """
SELECT g.genre_name, AVG(b.year_published) as avg_year
FROM genres g
JOIN books b ON g.genre_id = b.genre_id
GROUP BY g.genre_name;
"""
print(query_to_json(query_7))


def save_query_to_json_file(query, file_name):
    json_data = query_to_json(query)
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(json_data)

save_query_to_json_file(query_1, '5_1_result.json')
save_query_to_json_file(query_2, '5_2_result.json')
save_query_to_json_file(query_3, '5_3_result.json')
save_query_to_json_file(query_4, '5_4_result.json')
save_query_to_json_file(query_5, '5_5_result.json')
save_query_to_json_file(query_6, '5_6_result.json')
save_query_to_json_file(query_7, '5_7_result.json')

[
    {
        "book_title": "The Road",
        "year_published": 2006
    },
    {
        "book_title": "The Road",
        "year_published": 2006
    },
    {
        "book_title": "The Road",
        "year_published": 2006
    }
]
[
    {
        "genre_name": "Academic Fiction",
        "book_count": 0
    },
    {
        "genre_name": "Adventure",
        "book_count": 3
    },
    {
        "genre_name": "Alternate History",
        "book_count": 0
    },
    {
        "genre_name": "Autobiographical Novel",
        "book_count": 0
    },
    {
        "genre_name": "Autobiography",
        "book_count": 0
    },
    {
        "genre_name": "Biography",
        "book_count": 0
    },
    {
        "genre_name": "Chick Lit",
        "book_count": 0
    },
    {
        "genre_name": "Children's Fantasy",
        "book_count": 0
    },
    {
        "genre_name": "Children's Literature",
        "book_count": 10
    },
    {
        "genre_name": "Classic",
        "book_count"